In [2]:
# drafter, 결재선 이름, 조직도반영. 없는 것들의 시간은 00 00 00 으로

# 조직도도 반영 + actionDate 00:00:00인 sourceId 저장
import json
from bs4 import BeautifulSoup
from pathlib import Path
from datetime import datetime
import pandas as pd

# 설정
cmds_file = 'documents_2020.cmds'
output_file = 'documents_2020_fixed.cmds'
html_base_path = r'C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\결재'
CSV_FILE = '인사정보_부서코드추가.csv'
NEED_CRAWLING_FILE = 'need_crawling.json'  # ← 추가!

# CSV 읽기
print("📊 CSV 조직도 로드 중...\n")
df = pd.read_csv(CSV_FILE, encoding='utf-8-sig')

employee_dict = {}
for _, row in df.iterrows():
    employee_dict[row['사원명']] = {
        'emailId': row['ID'],
        'deptName': row['부서'],
        'empNo': row['사원번호'] if pd.notna(row['사원번호']) else '',
        'positionName': row['직위'] if pd.notna(row['직위']) else '',
        'deptCode': row['부서코드'] if pd.notna(row['부서코드']) else ''
    }
print(f"✅ 총 {len(employee_dict)}명 로드\n")

def find_html_file(source_id, base_path):
    """sourceId로 HTML 파일 찾기"""
    for html_file in Path(base_path).rglob('*.html'):
        if source_id in html_file.stem:
            return html_file
    return None

def extract_drafter_and_activities(html_path):
    """HTML에서 기안자, activities, createdAt 추출"""
    try:
        with open(html_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
        
        # 1. 기안자 이름
        drafter_name = ''
        drafter_th = soup.find('th', string=lambda s: s and '기안자' in s)
        if drafter_th:
            drafter_td = drafter_th.find_next_sibling('td')
            if drafter_td:
                bg01_div = drafter_td.find('div', class_='bg01')
                if bg01_div:
                    drafter_name = bg01_div.get_text(strip=True)
        
        # 2. 기안일
        created_at = None
        created_year = None
        created_th = soup.find('th', string=lambda s: s and '기안일' in s)
        if created_th:
            created_td = created_th.find_next_sibling('td')
            if created_td:
                bg01_div = created_td.find('div', class_='bg01')
                if bg01_div:
                    date_str = bg01_div.get_text(strip=True)
                    try:
                        dt = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
                        created_at = int(dt.timestamp() * 1000)
                        created_year = dt.year
                    except:
                        pass
        
        # 3. activities
        activities = []
        needs_crawling = []  # ← 추가! 크롤링 필요한 항목
        bg02_divs = soup.find_all('div', class_='bg02')
        
        for idx, bg02 in enumerate(bg02_divs):
            ul = bg02.find('ul')
            if ul:
                lis = ul.find_all('li')
                if len(lis) >= 2:
                    name = lis[0].get_text(strip=True)
                    action_type_text = lis[1].get_text(strip=True)
                    date_text = lis[2].get_text(strip=True) if len(lis) >= 3 else ''
                    
                    if name:
                        if '기안' in action_type_text:
                            action_type = 'DRAFT'
                        elif '합의' in action_type_text:
                            action_type = 'AGREEMENT'
                        else:
                            action_type = 'APPROVAL'
                        
                        # actionDate
                        action_date = None
                        if date_text and created_year:
                            try:
                                full_date = f"{created_year}-{date_text.replace('/', '-')}"
                                dt = datetime.strptime(full_date, "%Y-%m-%d")
                                action_date = int(dt.timestamp() * 1000)
                                
                                # 시간이 00:00:00이면 크롤링 필요 ← 추가!
                                needs_crawling.append({
                                    'index': idx,
                                    'name': name,
                                    'date_text': date_text,
                                    'action_type': action_type
                                })
                            except:
                                pass
                        
                        # 조직도 적용
                        if name in employee_dict:
                            position = employee_dict[name]['positionName']
                            dept = employee_dict[name]['deptName']
                            email = employee_dict[name]['emailId']
                            dept_code = employee_dict[name]['deptCode']
                        else:
                            # 퇴사자
                            position = ''
                            dept = ''
                            email = ''
                            dept_code = ''
                        
                        activities.append({
                            'positionName': position,
                            'deptName': dept,
                            'actionLogType': action_type,
                            'name': name,
                            'emailId': email,
                            'type': action_type,
                            'actionDate': action_date,
                            'deptCode': dept_code,
                            'actionComment': ''
                        })
        
        return drafter_name, activities, created_at, needs_crawling  # ← needs_crawling 추가!
        
    except Exception as e:
        print(f"  ⚠️ HTML 파싱 오류: {e}")
        return '', [], None, []

# cmds 파일 처리
print("📄 cmds 파일 처리 시작...\n")

crawling_needed = {}  # ← 추가! sourceId별 크롤링 필요 정보

with open(cmds_file, 'r', encoding='utf-8') as f_in, \
     open(output_file, 'w', encoding='utf-8') as f_out:
    
    fixed_count = 0
    total_count = 0
    
    for line in f_in:
        if line.startswith('addDocument '):
            json_str = line[12:].strip()
            doc = json.loads(json_str)
            
            drafter_name = doc.get('drafter', {}).get('name', '').strip()
            if not drafter_name:
                total_count += 1
                source_id = doc.get('sourceId')
                
                html_file = find_html_file(source_id, html_base_path)
                
                if html_file:
                    name, activities, created_at, needs_crawling = extract_drafter_and_activities(html_file)
                    
                    if name:
                        # drafter에 조직도 적용
                        if name in employee_dict:
                            doc['drafter']['name'] = name
                            doc['drafter']['positionName'] = employee_dict[name]['positionName']
                            doc['drafter']['deptName'] = employee_dict[name]['deptName']
                            doc['drafter']['emailId'] = employee_dict[name]['emailId']
                            doc['drafter']['deptCode'] = employee_dict[name]['deptCode']
                        else:
                            # 퇴사자
                            doc['drafter']['name'] = name
                            doc['drafter']['positionName'] = ''
                            doc['drafter']['deptName'] = ''
                            doc['drafter']['emailId'] = 'master'
                            doc['drafter']['deptCode'] = ''
                        
                        doc['activities'] = activities
                        doc['createdAt'] = created_at
                        fixed_count += 1
                        
                        # 크롤링 필요한 경우 저장 ← 추가!
                        if needs_crawling:
                            crawling_needed[source_id] = needs_crawling
                        
                        status = "현직" if name in employee_dict else "퇴사"
                        crawl_mark = " 🕐" if needs_crawling else ""
                        print(f"✅ {source_id}: '{name}'[{status}] + {len(activities)}개{crawl_mark}")
            
            f_out.write(f"addDocument {json.dumps(doc, ensure_ascii=False, separators=(',', ':'))}\n")
        else:
            f_out.write(line)

# 크롤링 필요 정보 저장 ← 추가!
if crawling_needed:
    with open(NEED_CRAWLING_FILE, 'w', encoding='utf-8') as f:
        json.dump(crawling_needed, f, ensure_ascii=False, indent=2)
    print(f"\n🕐 크롤링 필요한 문서: {len(crawling_needed)}건")
    print(f"   저장 위치: {NEED_CRAWLING_FILE}")

print(f"\n=== 처리 완료 ===")
print(f"총 비어있던 문서: {total_count}건")
print(f"수정 성공: {fixed_count}건")

📊 CSV 조직도 로드 중...

✅ 총 156명 로드

📄 cmds 파일 처리 시작...

✅ 2009497: '김은미'[퇴사] + 3개 🕐
✅ 2009500: '윤상호'[현직] + 3개 🕐
✅ 2009499: '김은미'[퇴사] + 3개 🕐
✅ 2009498: '김은미'[퇴사] + 3개 🕐
✅ 2009507: '원진희'[현직] + 3개 🕐
✅ 2009508: '현광섭'[퇴사] + 2개 🕐
✅ 2009506: '편영수'[현직] + 2개 🕐
✅ 2009509: '김승범'[퇴사] + 3개 🕐
✅ 2009510: '양기훈'[퇴사] + 4개 🕐
✅ 2009511: '이형균'[퇴사] + 4개 🕐
✅ 2009512: '전수정'[퇴사] + 4개 🕐
✅ 2009513: '김문철'[퇴사] + 3개 🕐
✅ 2009514: '김문철'[퇴사] + 3개 🕐
✅ 2009515: '이송연'[퇴사] + 3개 🕐
✅ 2009519: '권민선'[퇴사] + 5개 🕐
✅ 2009518: '강정례'[퇴사] + 3개 🕐
✅ 2009517: '강정례'[퇴사] + 3개 🕐
✅ 2009523: '김형환'[퇴사] + 6개 🕐
✅ 2009524: '이형균'[퇴사] + 3개 🕐
✅ 2009525: '강정례'[퇴사] + 4개 🕐
✅ 2009526: '장광선'[퇴사] + 4개 🕐
✅ 2009536: '김문철'[퇴사] + 4개 🕐
✅ 2009531: '권민선'[퇴사] + 4개 🕐
✅ 2009530: '권민선'[퇴사] + 4개 🕐
✅ 2009532: '권민선'[퇴사] + 4개 🕐
✅ 2009533: '권민선'[퇴사] + 4개 🕐
✅ 2009527: '임현준'[현직] + 5개 🕐
✅ 2009529: '원진희'[현직] + 4개 🕐
✅ 2009535: '강정례'[퇴사] + 6개 🕐
✅ 2009528: '편영수'[현직] + 4개 🕐
✅ 2009538: '이성현'[퇴사] + 4개 🕐
✅ 2009544: '윤상호'[현직] + 3개 🕐
✅ 2009540: '권민선'[퇴사] + 4개 🕐
✅ 2009543: '윤상호'[현직] + 4